In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('datasets_heart.csv')
df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [ ]:
df = df.drop(columns=['education'])
df.head()

,male,age,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [ ]:
df.shape

(4240, 15)

In [ ]:
df.isnull().sum()

,0
male,0
age,0
currentSmoker,0
cigsPerDay,29
BPMeds,53
prevalentStroke,0
prevalentHyp,0
diabetes,0
totChol,50
sysBP,0


**Fill the missing values**

In [ ]:
#for train the binary values in the data
bin_cols = ["male", "currentSmoker", "prevalentStroke", "prevalentHyp", "diabetes"]
for col in bin_cols:
  mode_val = df[col].mode()[0]
  df[col] = df[col].fillna(mode_val)

In [ ]:
#for the numeric feautures
num_cols = ["cigsPerDay", "BPMeds", "totChol", "BMI", "heartRate", "glucose"]
for col in num_cols:
  median_val = df[col].median()
  df[col] = df[col].fillna(median_val)

In [ ]:
df.isnull().sum()

,0
male,0
age,0
currentSmoker,0
cigsPerDay,29
BPMeds,53
prevalentStroke,0
prevalentHyp,0
diabetes,0
totChol,50
sysBP,0


In [ ]:
df['TenYearCHD'].value_counts()

,count
TenYearCHD,
0,3596
1,644


In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['TenYearCHD'] == 0]
df_minority = df[df['TenYearCHD'] == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True,     # Sample with replacement
                                 n_samples=len(df_majority),    # To match majority class
                                 random_state=42) # Reproducible results

# Combine majority class with upsampled minority class
df_balanced = pd.concat([df_majority, df_minority_upsampled])

In [ ]:

df_balanced['TenYearCHD'].value_counts()

,count
TenYearCHD,
0,3596
1,3596


In [ ]:
from sklearn.model_selection import train_test_split

X = df_balanced.drop(columns=['TenYearCHD'])
y = df_balanced['TenYearCHD']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42  )


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_test_scaled

array([[-0.94172615, -0.97246749,  0.97442734, ...,  0.48593329,
        -0.08995259,  0.61151248],
       [ 1.06187982, -0.16224222, -1.02624378, ..., -0.99823818,
         0.32721252, -0.40552742],
       [ 1.06187982,  0.64798304,  0.97442734, ..., -0.02520317,
        -1.17458187,  0.04316665],
       ...,
       [-0.94172615, -0.85672102, -1.02624378, ..., -1.0404421 ,
        -1.34144792,         nan],
       [-0.94172615,  0.64798304,  0.97442734, ..., -0.22918882,
         1.57870785, -0.25596273],
       [-0.94172615,  0.06925071, -1.02624378, ..., -0.36283459,
        -1.50831396,  1.14994537]])

**Train Test split**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled,y_train)
y_pred = rf_model.predict(X_test_scaled,)
accuracy_score = accuracy_score(y_test,y_pred)
print(accuracy_score)

0.9777623349548298


**Scaling**

In [ ]:
# Initialize StandardScaler
scaler = StandardScaler()

# Fit scaler to training data and transform both training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Training the model in 10 diff ways**

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd  # Import pandas for data handling
import numpy as np  # Import numpy for numerical operations

# ... (your existing code) ...

# Define a list of classifiers
classifiers = [
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    LogisticRegression(),
    SVC(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    GaussianNB(),
    XGBClassifier()
]

# Create a dictionary to store the results
results = {}

# Train and evaluate each classifier
for clf in classifiers:
    clf_name = clf.__class__.__name__

    # Impute missing values if necessary (before fitting)
    # This will replace NaNs with the median for numeric features
    # and the mode for categorical features

    # For X_train_scaled
    X_train_scaled_imputed = pd.DataFrame(X_train_scaled).fillna(pd.DataFrame(X_train_scaled).median()).values
    # For X_test_scaled
    X_test_scaled_imputed = pd.DataFrame(X_test_scaled).fillna(pd.DataFrame(X_test_scaled).median()).values

    clf.fit(X_train_scaled_imputed, y_train)  # Use imputed data for training
    y_pred = clf.predict(X_test_scaled_imputed)  # Use imputed data for prediction

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{clf_name} Accuracy: {accuracy}")

    # Classification report
    print(f"Classification Report for {clf_name}:")
    print(classification_report(y_test, y_pred))

    # Confusion matrix
    print(f"Confusion Matrix for {clf_name}:")
    print(confusion_matrix(y_test, y_pred))
    print("="*50)

RandomForestClassifier Accuracy: 0.9763724808895066
Classification Report for RandomForestClassifier:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       735
           1       0.96      0.99      0.98       704

    accuracy                           0.98      1439
   macro avg       0.98      0.98      0.98      1439
weighted avg       0.98      0.98      0.98      1439

Confusion Matrix for RandomForestClassifier:
[[706  29]
 [  5 699]]
AdaBoostClassifier Accuracy: 0.6518415566365532
Classification Report for AdaBoostClassifier:
              precision    recall  f1-score   support

           0       0.68      0.61      0.64       735
           1       0.63      0.70      0.66       704

    accuracy                           0.65      1439
   macro avg       0.65      0.65      0.65      1439
weighted avg       0.65      0.65      0.65      1439

Confusion Matrix for AdaBoostClassifier:
[[447 288]
 [213 491]]
GradientBoostingCl

**Selecting the best model**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Instantiate the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the RandomForestClassifier
rf_classifier.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred_rf = rf_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Classifier Accuracy:", accuracy_rf)

# Classification report
print("Classification Report for Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))

# Confusion matrix
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test, y_pred_rf))

Random Forest Classifier Accuracy: 0.9749826268241835
Classification Report for Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       735
           1       0.96      0.99      0.97       704

    accuracy                           0.97      1439
   macro avg       0.98      0.98      0.97      1439
weighted avg       0.98      0.97      0.97      1439

Confusion Matrix for Random Forest Classifier:
[[704  31]
 [  5 699]]


In [ ]:
#test 01
print("predicted_class",rf_classifier.predict(X_test_scaled[10].reshape(1,-1))[0])
print("actual_class",y_test.iloc[10])

predicted_class 0
actual_class 0


In [ ]:
#test 02
print("predicted_class",rf_classifier.predict(X_test_scaled[200].reshape(1,-1))[0])
print("actual_class",y_test.iloc[200])

predicted_class 1
actual_class 1


In [ ]:
import pickle
pickle.dump(rf_classifier,open('rf_classifiesr.pkl','wb'))
pickle.dump(scaler,open('scaler.pkl','wb'))


**Predictive System**

In [ ]:
import numpy as np

def predict(model, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose):
    # Encode categorical variables
    male_encoded = 1 if male.lower() == "male" else 0
    currentSmoker_encoded = 1 if currentSmoker.lower() == "yes" else 0
    BPMeds_encoded = 1 if BPMeds.lower() == "yes" else 0
    prevalentStroke_encoded = 1 if prevalentStroke.lower() == "yes" else 0
    prevalentHyp_encoded = 1 if prevalentHyp.lower() == "yes" else 0
    diabetes_encoded = 1 if diabetes.lower() == "yes" else 0

    # Prepare features array
    features = np.array([[male_encoded, age, currentSmoker_encoded, cigsPerDay, BPMeds_encoded, prevalentStroke_encoded, prevalentHyp_encoded, diabetes_encoded, totChol, sysBP, diaBP, BMI, heartRate, glucose]])

    # scalling
    scaled_features = scaler.transform(features)

    # predict by model
    result = model.predict(scaled_features)

    return result[0]

In [ ]:
# test 1:
male = "female"
age = 56.00
currentSmoker = "yes"
cigsPerDay = 3.00
BPMeds = "no"
prevalentStroke = "no"
prevalentHyp = "yes"
diabetes = 'no'
totChol = 285.00
sysBP = 145.00
diaBP = 100.00
BMI = 30.14
heartRate = 80.00
glucose = 86.00


result = predict(rf_classifier, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose)


if result == 1:
    print("The Patient has Heart Diseas")
else:
    print("The Patiennt has No Heart Deseas")

The Patiennt has No Heart Deseas


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
male = 'female'
age = 63.0
currentSmoker = 'yes'
cigsPerDay = 3.0
BPMeds = 'no'
prevalentStroke = 'no'
prevalentHyp = 'yes'
diabetes = 'no'
totChol = 267.0
sysBP = 156.5
diaBP = 92.5
BMI = 27.1
heartRate = 60.0
glucose = 79.0
result = 1.0



result = predict(rf_classifier, scaler, male, age, currentSmoker, cigsPerDay, BPMeds, prevalentStroke, prevalentHyp, diabetes, totChol, sysBP, diaBP, BMI, heartRate, glucose)


if result == 1:
    print("The Patient has Heart Diseas")
else:
    print("The Patiennt has No Heart Deseas")

The Patient has Heart Diseas


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import sklearn
sklearn.__version__

'1.6.1'